In [ ]:
import cx_Oracle
import pandas as pd

conn =  cx_Oracle.connect(user="pc",password="p201404",dsn="siatchdesa")
query = "select * from PC.L_VM_COMPLETA WHERE PERIODO <= 201910" 
df = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM

# Para concatenar todos los chunks en un solo DataFrame (si tienes suficiente RAM)
df_full = pd.concat(df, ignore_index=True)
conn.close()
print(df_full.shape)
# Imprimir las primeras filas del DataFrame completo
print(df_full.head())
# Imprimir los tipos de datos de las columnas del DataFrame completo
print(df_full.dtypes)
# Imprimir el número de filas y columnas del DataFrame completo
print(f"Número de filas: {df_full.shape[0]}, Número de columnas: {df_full.shape[1]}")

In [ ]:
# Usando este código como base para un modelo de regresión con LightGBM
# agregar hojas de LightGBM

import lightgbm as lgb

# Variables predictoras y objetivo
X = df_full[['PERIODO','ANIO','MES','TRIMESTRE','ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','SKU_SIZE','CUSTOMER_ID','PRODUCT_ID','PLAN_PRECIOS_CUIDADOS',
'TN','STOCK_FINAL','MEDIA_MOVIL_3M','DESVIO_MOVIL_3M','TN_LAG_01','TN_LAG_02','TN_LAG_03','TN_LAG_06','TN_LAG_12',
'TN_DELTA_01','TN_DELTA_02','TN_DELTA_03','TN_DELTA_06','TN_DELTA_12']]
y = df_full['CLASE_DELTA']

# Variables categóricas
categorical_features = ['ANIO','MES','TRIMESTRE','ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','SKU_SIZE','CUSTOMER_ID','PRODUCT_ID','PLAN_PRECIOS_CUIDADOS']
# Convertir las variables categóricas a tipo 'category'
for col in categorical_features:
    df_full[col] = df_full[col].astype('category')
# Asegurarse de que las variables predictoras y objetivo estén en el formato correcto

# definir las variables predictoras y la variable objetivo indicando al modelo que las variables categóricas son de tipo 'category'


In [ ]:

# Crear el dataset de LightGBM
train_data = lgb.Dataset(X, label=y)
# Especificar las variables categóricas
train_data = lgb.Dataset(X, label=y, categorical_feature=categorical_features)
# Imprimir información del dataset de LightGBM
print("Dataset de LightGBM creado con las variables categóricas especificadas.")

# Definir parámetros para regresión
params = { 
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'num_leaves': 255,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1 }

# Entrenar el modelo
model = lgb.train(params, train_data, num_boost_round=10000)

print("Modelo de regresión entrenado con todo el dataset.")

In [ ]:
# Guardo el modelo entrenado
model.save_model('modelo_regresion_lgbm_lags_delta_promedio_desvio.txt')

In [ ]:
# Cargo los datos sobre los que quiero hacer predicciones

conn =  cx_Oracle.connect(user="pc",password="p201404",dsn="siatchdesa")
query = "select * from L_VM_COMPLETA where periodo = 201912" 
df_pred = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM

# Para concatenar todos los chunks en un solo DataFrame (si tienes suficiente RAM)
df_pred_full = pd.concat(df_pred, ignore_index=True)
conn.close()

In [ ]:
# Variables categóricas
categorical_features = ['ANIO','MES','TRIMESTRE','ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','SKU_SIZE','CUSTOMER_ID','PRODUCT_ID','PLAN_PRECIOS_CUIDADOS']
# Convertir las variables categóricas a tipo 'category'
for col in categorical_features:
    df_pred_full[col] = df_pred_full[col].astype('category')


# Con el modelo entrenado, hacemos predicciones 
X_pred = df_pred_full[['PERIODO','ANIO','MES','TRIMESTRE','ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','SKU_SIZE','CUSTOMER_ID','PRODUCT_ID','PLAN_PRECIOS_CUIDADOS',
'TN','STOCK_FINAL','MEDIA_MOVIL_3M','DESVIO_MOVIL_3M','TN_LAG_01','TN_LAG_02','TN_LAG_03','TN_LAG_06','TN_LAG_12',
'TN_DELTA_01','TN_DELTA_02','TN_DELTA_03','TN_DELTA_06','TN_DELTA_12']]


predictions = model.predict(X_pred)
# Agregar las predicciones al DataFrame original
df_pred_full['PREDICCIONES_DELTA'] = predictions
# Agregar la columna PREDICCIONES que es TN + PREDICCIONES_DELTA
df_pred_full['PREDICCIONES'] = df_pred_full['TN'] + df_pred_full['PREDICCIONES_DELTA']
# Imprimir las primeras filas del DataFrame con las predicciones
print(df_pred_full.head())
# Guardar el DataFrame con las predicciones en un archivo CSV
df_pred_full.to_csv('predicciones.csv', index=False)
# Imprimir el número de filas y columnas del DataFrame con las predicciones
print(f"Número de filas: {df_pred_full.shape[0]}, Número de columnas: {df_pred_full.shape[1]} con predicciones.")



In [ ]:
# Con el DataFrame de predicción, actualizamos la base de datos
# el criterio es actualizar la tabla L_VM_COMPLETA_PREDICCIONES con las nuevas predicciones
# la columnna PREDICCIONES se debe actualizar con los nuevos valores
# la clave primaria es (PERIODO, CUSTOMER_ID, PRODUCT_ID)
# Hacer commit cada 10000 filas para evitar problemas de memoria

# Conectar a la base de datos para actualizar los datos de predicción
conn = cx_Oracle.connect(user="pc", password="p201404", dsn="siatchdesa")
# Crear un cursor para ejecutar las actualizaciones
cursor = conn.cursor()
update_query = """
    UPDATE L_VM_COMPLETA_PREDICCIONES
    SET PREDICCION = NULL
"""
cursor.execute(update_query)
# Hacer commit para aplicar el cambio de NULL
conn.commit()


In [ ]:


# Imprimir mensaje de inicio de actualización
print("Iniciando actualización de la tabla L_VM_COMPLETA_PREDICCIONES con las nuevas predicciones.")

# Iterar sobre las filas del DataFrame con las predicciones
for index, row in df_pred_full.iterrows():
    periodo = row['PERIODO']
    customer_id = row['CUSTOMER_ID']
    product_id = row['PRODUCT_ID']
    prediccion = row['PREDICCIONES']
    
    # Actualizar la tabla L_DATOS_PREDICCION con la nueva predicción
    update_query = """
        UPDATE L_VM_COMPLETA_PREDICCIONES
        SET PREDICCION = :prediccion
        WHERE PERIODO = :periodo AND CUSTOMER_ID = :customer_id AND PRODUCT_ID = :product_id
    """
    cursor.execute(update_query, {'prediccion': prediccion, 'periodo': periodo, 'customer_id': customer_id, 'product_id': product_id})  
    # Hacer commit cada 10000 filas para evitar problemas de memoria
    if index % 10000 == 0:
        conn.commit()
        print(f"Actualizadas {index} filas de L_VM_COMPLETA_PREDICCIONES con las nuevas predicciones.")
# Confirmar los cambios en la base de datos
conn.commit()
# Cerrar el cursor y la conexión
cursor.close()
conn.close()
# Imprimir mensaje de finalización
print("Actualización de la tabla  completada con las nuevas predicciones.")
